In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%matplotlib widget

In [3]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

### Reference: Influence of hole shape on sound absorption of underwater anechoic layers
#### https://www.sciencedirect.com/science/article/abs/pii/S0022460X1830227X

### Import pre-defined class and function

In [18]:
%run -i hole_shape_sound_absorption.py

### Define the determinant of the coefficient matrix of eq(14) in the paper

In [5]:
determinant, mat = determinant_from_matrix()
mat

Matrix([
[                                                                                                              -sqrt(-kz**2 + omega**2/cl**2)*besselj(1, b*sqrt(-kz**2 + omega**2/cl**2)),                                                                                                               -sqrt(-kz**2 + omega**2/cl**2)*bessely(1, b*sqrt(-kz**2 + omega**2/cl**2)),                                                                                      -I*kz*sqrt(-kz**2 + omega**2/ct**2)*besselj(1, b*sqrt(-kz**2 + omega**2/ct**2)),                                                                                      -I*kz*sqrt(-kz**2 + omega**2/ct**2)*bessely(1, b*sqrt(-kz**2 + omega**2/ct**2))],
[(-2*mu*(-kz**2 + omega**2/cl**2) - lambda*omega**2/cl**2)*besselj(0, a*sqrt(-kz**2 + omega**2/cl**2)) + 2*mu*sqrt(-kz**2 + omega**2/cl**2)*besselj(1, a*sqrt(-kz**2 + omega**2/cl**2))/a, (-2*mu*(-kz**2 + omega**2/cl**2) - lambda*omega**2/cl**2)*bessely(0, a*sqrt(-kz**2 + omega**2/cl**

### Define the frequency array by np.arange

In [6]:
# np.arange?

In [7]:
# frequency_array = np.arange(20, 10021, 100)
# frequency_array

## Define an object of sound_performance for calculating sound absorption
### The default values are listed as below (all in SI units)

Material of anechoic layer: rubber

Young's modulus of rubber (GPa): 0.14

Loss factor of rubber 0.23

Poisson ratio of rubber 0.49

Density of rubber (kg/m3): 1100

Density of air (kg/m3): 1.21

Shape: cone

p = 4 mm

q = 8 mm

lh = 40 mm

cell_radius = 15 mm

num_segments = 100

Density of water (kg/m3): 998

Sound speed of water (m/s): 1483

#### https://en.wikipedia.org/wiki/Lam%C3%A9_parameters

In [19]:
frequency_array = np.arange(2, 10052, 50)
frequency_array.shape

(201,)

### Example 01: default case

In [20]:
# frequency_array = np.asarray([20, 4620])
# cone_4_8_4 = sound_performance(determinant, frequency_array)
# cone_4_8 = sound_performance(determinant, frequency_array)
cone_0_3 = sound_performance(determinant, frequency_array)

In [21]:
print(f'{cone_0_3.material = }')
print(f'{cone_0_3.shape = }')
print(f'{cone_0_3.Young = }')
print(f'{cone_0_3.p_hole = }')
print(f'{cone_0_3.q_hole = }')

cone_0_3.material = 'rubber'
cone_0_3.shape = 'cone'
cone_0_3.Young = 140000000.0
cone_0_3.p_hole = 0.004
cone_0_3.q_hole = 0.008


In [22]:
# cone_4_8_4.segments = 50
# cone_4_8.segments = 50
cone_0_3.segments = 50

In [23]:
# cone_4_8_5.Young = 1.4e7

In [24]:
cone_0_3.p_hole = 0.000

In [25]:
cone_0_3.q_hole = 0.003

In [26]:
# cone_4_8.Young = 150
# cone_4_8.shape = 'horn'

In [27]:
ai, _ = cone_0_3.effective_radius()
ai

array([0.00000000e+00, 6.12244898e-05, 1.22448980e-04, 1.83673469e-04,
       2.44897959e-04, 3.06122449e-04, 3.67346939e-04, 4.28571429e-04,
       4.89795918e-04, 5.51020408e-04, 6.12244898e-04, 6.73469388e-04,
       7.34693878e-04, 7.95918367e-04, 8.57142857e-04, 9.18367347e-04,
       9.79591837e-04, 1.04081633e-03, 1.10204082e-03, 1.16326531e-03,
       1.22448980e-03, 1.28571429e-03, 1.34693878e-03, 1.40816327e-03,
       1.46938776e-03, 1.53061224e-03, 1.59183673e-03, 1.65306122e-03,
       1.71428571e-03, 1.77551020e-03, 1.83673469e-03, 1.89795918e-03,
       1.95918367e-03, 2.02040816e-03, 2.08163265e-03, 2.14285714e-03,
       2.20408163e-03, 2.26530612e-03, 2.32653061e-03, 2.38775510e-03,
       2.44897959e-03, 2.51020408e-03, 2.57142857e-03, 2.63265306e-03,
       2.69387755e-03, 2.75510204e-03, 2.81632653e-03, 2.87755102e-03,
       2.93877551e-03, 3.00000000e-03])

In [28]:
## Find roots of wavenumber in the equation of determinant
# cone_4_8_4.wavenumer_array, cone_4_8_4.failed_root = cone_4_8_4.axial_wavenumber_array()
# cone_4_8.wavenumer_array, cone_4_8.failed_root = cone_4_8.axial_wavenumber_array()
cone_0_3.wavenumer_array, cone_0_3.failed_root = cone_0_3.axial_wavenumber_array()

Solving wavenumber in determinant for shape = cone, p = 0.0, q = 0.003, Young's = 140000000.0


Solving for all frequencies:   0%|          | 0/201 [00:00<?, ?it/s]

  ... working at frequency = 2.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

  ... working at frequency = 52.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

  ... working at frequency = 102.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

...

  ... working at frequency = 9802.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

  ... working at frequency = 9852.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

  ... working at frequency = 9902.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

  ... working at frequency = 9952.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

  ... working at frequency = 10002.0 Hz:   0%|          | 0/50 [00:00<?, ?it/s]

In [30]:
cone_0_3.failed_root

array([], shape=(201, 0), dtype=float64)

In [ ]:
cone_6_3.__dict__.keys()

In [ ]:
# ## Calculate sound absorption of cone_4_8
# cone_4_8.absorption_frequency()

In [ ]:
cone_6_3.omega_array().shape

In [ ]:
## Save the data
fp = '/Users/chenghunglin/Documents/'
fn = 'cone_6_3.xlsx'
# fn5 = 'cone_4_8_5.xlsx'
cone_6_3.save_data(fp, fn)
# cone_4_8_5.save_data(fp, fn5)

In [ ]:
plt.figure()
plt.plot(cone_6_3.frequency_array, cone_6_3.absorption_frequency(), label='cone, p=6, q=3, Young=0.14 Gpa')
plt.plot(cone_4_8.frequency_array, cone_4_8.absorption_frequency(), label='cone, p=4, q=8, Young=0.14 Gpa')
# plt.plot(cone_4_8_3.frequency_array, cone_4_8_3.absorption_frequency(), '--', color='orange', label='cone, p=4, q=8, Young=0.014 Gpa')
# plt.plot(horn_1_11.frequency_array, horn_1_11.absorption_frequency(), label='horn, p=1, q=11')
plt.legend()

### Example 02: change shape, p, q

In [ ]:
horn_1_11 = sound_performance(determinant, frequency_array)

In [ ]:
horn_1_11.shape = 'horn'
horn_1_11.p_hole = 1e-3
horn_1_11.q_hole = 11e-3

In [ ]:
## Find roots of wavenumber in the equation of determinant
horn_1_11.wavenumer_array, horn_1_11.failed_root = horn_1_11.axial_wavenumber_array()

In [ ]:
## Save the data
fp = '/Users/chenghunglin/Documents/'
fn = 'horn_1_11.xlsx'
horn_1_11.save_data(fp, fn)

## Plot data to compare with the paper

In [ ]:
plt.figure()
plt.plot(cone_4_8.frequency_array, cone_4_8.absorption_frequency(), label='cone, p=4, q=8, Young=0.14 Gpa')
plt.plot(cone_4_8_1.frequency_array, cone_4_8_1.absorption_frequency(), label='cone, p=4, q=8, Young=0.014 Gpa')
plt.plot(cone_4_8_3.frequency_array, cone_4_8_3.absorption_frequency(), '--', color='orange', label='cone, p=4, q=8, Young=0.014 Gpa')
# plt.plot(horn_1_11.frequency_array, horn_1_11.absorption_frequency(), label='horn, p=1, q=11')
plt.legend()

In [ ]:
plt.figure()
plt.plot(cone_4_8.frequency_array, cone_4_8.absorption_frequency(), color='blue', label='cone, p=4, q=8, Young=0.14 Gpa, step=200Hz')
plt.plot(cone_4_8_4.frequency_array, cone_4_8_4.absorption_frequency(), '--', color='blue',label='cone, p=4, q=8, Young=0.14 Gpa, step=25Hz')
plt.plot(cone_4_8_1.frequency_array, cone_4_8_1.absorption_frequency(), color='orange', label='cone, p=4, q=8, Young=0.014 Gpa, step=200Hz')
plt.plot(cone_4_8_5.frequency_array, cone_4_8_5.absorption_frequency(), '--', color='orange', label='cone, p=4, q=8, Young=0.014 Gpa, step=25Hz')
# plt.plot(horn_1_11.frequency_array, horn_1_11.absorption_frequency(), label='horn, p=1, q=11')
legend_properties = {'weight':'regular', 'size':8}    
plt.legend(prop=legend_properties, loc='upper left') #, bbox_to_anchor=(1, 0.5))
# plt.legend()